In [ ]:
# 02_analise_exploratoria_dados.ipynb

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, hour, dayofweek, to_timestamp

spark = SparkSession.builder \
    .appName("ChicagoCrimeEDA") \
    .getOrCreate()

# Ler dados do ficheiro Parquet (amostra)
df = spark.read.parquet("../dados/chicago_crime_small.parquet")

# Converter colunas de data para timestamp
df = df.withColumn("Date", to_timestamp("Date", "MM/dd/yyyy hh:mm:ss a")) \
       .withColumn("Updated On", to_timestamp("Updated On", "MM/dd/yyyy hh:mm:ss a"))

# Verificar esquema convertido
df.printSchema()

# Estatísticas descritivas gerais
df.describe().show()

# Frequência de crimes por tipo
print("\nFrequência dos crimes por tipo:")
df.groupBy("Primary Type").count().orderBy(desc("count")).show(10, truncate=False)

# Frequência de arrestos
print("\nFrequência de crimes com e sem arresto:")
df.groupBy("Arrest").count().show()

# Frequência por localização (top 10)
print("\nTop 10 locais com mais crimes:")
df.groupBy("Location Description").count().orderBy(desc("count")).show(10, truncate=False)

# Frequência por hora do dia
print("\nFrequência dos crimes por hora do dia:")
df.withColumn("Hour", hour("Date"))\
  .groupBy("Hour").count().orderBy("Hour").show(24)

# Frequência por dia da semana
print("\nFrequência dos crimes por dia da semana:")
df.withColumn("DayOfWeek", dayofweek("Date"))\
  .groupBy("DayOfWeek").count().orderBy("DayOfWeek").show(7)

# Frequência por tipo de crime com arresto
print("\nTop 10 tipos de crimes que resultaram em arresto:")
df.filter(col("Arrest") == True)\
  .groupBy("Primary Type").count().orderBy(desc("count")).show(10, truncate=False)

spark.stop()